# Garbage Detection 🗞🍌🧃🥫


## Install Requirements

In [ ]:
# yolov5 repo
yolov5_cloned = True
if not yolov5_cloned:
    !git clone https://github.com/ultralytics/yolov5  # clone yolov5repo
yolov5_requirements_installed = True
if not yolov5_requirements_installed:
    %cd yolov5
    %pip install -qr requirements.txt
    %cd ..

# other requirements
requirements_installed = True
if not requirements_installed:
    %pip install -q requirements.txt

In [ ]:
import torch
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

## Download Dataset


In [ ]:
dataset_downloaded = True
if not dataset_downloaded:
    from roboflow import Roboflow
    rf = Roboflow(api_key="ZG73C7nrLF5yXM6t29E3",model_format="yolov5", notebook="ultralytics")
    !mkdir data
    !cd data
    # set up environment
    import os
    os.environ["DATASET_DIRECTORY"] = os.getcwd()+"/data"
    rf = Roboflow(api_key="ZG73C7nrLF5yXM6t29E3")
    project = rf.workspace("sappia").project("hri-ra-mtc9l")
    dataset = project.version(3).download("yolov5")
    %cd ../yolov5

## Train ⚡ 
> Finetuning YOLOv5

Here, we are able to pass a number of arguments:
- **img:** define input image size
- **batch:** determine batch size
- **epochs:** define the number of training epochs. (Note: often, 3000+ are common here!)
- **data:** Our dataset location is saved in the `dataset.location`
- **weights:** specify a path to weights to start transfer learning from. Here we choose the generic COCO pretrained checkpoint.
- **cache:** cache images for faster training

In [ ]:
%cd yolov5
!python train.py --img 640 --batch 32 --epochs 20 --data ../data/HRI-RA-3/data.yaml --weights yolov5m.pt --cache

## Test 📏
> Only for statistics purposes to add to the report.

In [ ]:
%cd yolov5
!python val.py --weights ../model/best.pt --data ../data/HRI-RA-3/data.yaml --img 640

### FP16 half-precision

In [ ]:
!python val.py --weights ../model/best.pt --data ../data/HRI-RA-3/data.yaml --img 640 --half

### Post Pruning ✂

We repeat the above test with a pruned model by using the *torch_utils.prune()* command. We create a copy of *val.py* file named *val_prune.py* to post-prune the model.

Add this lines of code to the new created *val_prune.py* file:
<a href="https://imgur.com/SoR1aJn"><img src="https://i.imgur.com/SoR1aJn.png" title="source: imgur.com" /></a>

In [ ]:
!python val_prune.py --weights ../model/best.pt --data ../data/HRI-RA-3/data.yaml --img 640 --half